Chrome is a memory hog, this notebook works best in other browsers.

In [4]:
import pickle
import sys
sys.path.append('python')
import clusterOutliers
with open('data/output/Q8_sample.coo','rb') as file:
    Q8_coo = pickle.load(file)

In [4]:
import numpy as np
from astropy.io import fits
import pandas as pd

data = Q8_coo.data
Quarter = 8
pca = Q8_coo.reductions['PCA90']
x = pca.iloc[:,0]
y = pca.iloc[:,1]
files = data.index
labels = np.array(['KIC '+str(int(lc[4:13])) for lc in files])

In [3]:
Q8_coo.scores

,k1_1000x10,DB_outliers
kplr011241837-2011073133259_llc.fits,0.003211,0
kplr011821279-2011073133259_llc.fits,0.003997,0
kplr006342144-2011073133259_llc.fits,0.002002,0
kplr010355036-2011073133259_llc.fits,0.003108,0
kplr007757721-2011073133259_llc.fits,0.001226,0
...,...,...
kplr011721462-2011073133259_llc.fits,0.001952,0
kplr007499535-2011073133259_llc.fits,0.001459,0
kplr008960179-2011073133259_llc.fits,0.006952,0
kplr010861822-2011073133259_llc.fits,0.001356,0


In [5]:
scores = Q8_coo.scores['k1_1000x10']
dblabels = Q8_coo.scores['DB_outliers']
dblabels_as_string = np.array(dblabels,dtype=str)

In [10]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [51]:
dblabels           = Q8_coo.scores['DB_outliers'][:1000]
dblabels_as_string = np.array(dblabels,dtype=str)
scores             = Q8_coo.scores['k1_1000x10'][:1000]

dbcolors           = colors_for_plot(dblabels,'color_blind')
lin_colors         = colors_for_plot(-scores,'viridis')
log_colors         = colors_for_plot(-np.log(scores),'viridis')

cmap_dict = {
    "Outliers"        : [to_hex(c) for c in dbcolors],
    "Scores - Linear" : [to_hex(c) for c in lin_colors],
    "Scores - Log"    : [to_hex(c) for c in log_colors],
}


In [61]:
len(cmap_dict['Scores - Log'])

1000

In [11]:
import pickle
import sys
sys.path.append('python')
import clusterOutliers
import numpy as np
from quarterTools import read_kepler_curve, pca_red, colors_for_plot
import pandas as pd
from lightkurve import search_lightcurvefile
from matplotlib.colors import to_hex

from bokeh.io import curdoc
from bokeh.plotting import figure
from bokeh.layouts import row,gridplot,column
from bokeh.models import ColumnDataSource, Circle
from bokeh.models.widgets import Button, Select
from bokeh.events import Tap
from bokeh.palettes import Colorblind4,Category10
from bokeh.transform import factor_cmap, linear_cmap, log_cmap

def modify_doc(doc):
    
    ##### Data Import #####
    with open('data/output/Q8_sample.coo','rb') as file:
        Q8_coo = pickle.load(file)

    data               = Q8_coo.data[:1000]
    Quarter            = 8
    pca                = Q8_coo.reductions['PCA90']
    x                  = pca.iloc[:1000,0]
    y                  = pca.iloc[:1000,1]
    files              = data.index
    labels             = np.array(['KIC '+str(int(lc[4:13])) for lc in files])
    dblabels           = Q8_coo.scores['DB_outliers'][:1000]
    dblabels_as_string = np.array(dblabels,dtype=str)
    scores             = Q8_coo.scores['k1_1000x10'][:1000]
    dbcolors           = colors_for_plot(dblabels,'color_blind')
    lin_colors         = colors_for_plot(-scores,'viridis')
    log_colors         = colors_for_plot(-np.log(scores),'viridis')
    ##### Bokeh Setup #####
    # Tools to use
    TOOLS     = "pan,wheel_zoom,reset,tap,box_select,poly_select"
    palette   = Colorblind4[0], Colorblind4[1], Colorblind4[3] # Color palette for outlier cmap
    alphas    = {1:1,2:.9,3:.8,4:.75,5:.7,6:.65,7:.6,8:.55,9:.5,10:.5}
    cmap_dict = {
        "Base"            : palette[0],
        "Outliers"        : [to_hex(c) for c in dbcolors],
        "Scores - Linear" : [to_hex(c) for c in lin_colors],
        "Scores - Log"    : [to_hex(c) for c in log_colors],
    }

    def update_colors(attrname, old, new):
        """ 
        Bokeh was being uncooperative with it's color mapper combined with the selection tool.
        I could get the colors to change when a different color selection was made,
        but it wouldn't update the color mapping to selected data. Whenever switching to a new
        color selection, selecting data would revert to the previous color mapping except where
        colors are given explicitly. So, I'm defining the colors explicitly using matplotlib
        instead of using the more convenient Bokeh mapper function which I can't find a way to save the 
        mapping as an array for the life of me. It's clunky and I hate it but it works.
        """
        if select.value == "Base":
            s1.data['colors'] = [cmap_dict[select.value]]*len(s1.data['x'])
        else:
            s1.data['colors'] = cmap_dict[select.value]
    def update_plot():
        try:
            ind = s1.selected.indices[0] # can only plot one at a time
        except:
            ind = 0
            
        lc_path = 'Plots/'+s1.data['files'][ind][:-5]+'.png'
        lc_image = plc.image_url(url=[lc_path],x=0,y=0,w=10,h=3,anchor="bottom_left")
        
    # Bokeh data
    # create a column data source for the plots to share
    s1 = ColumnDataSource(data={
        'x'     : x,
        'y'     : y,
        'desc'  : labels,
        'files' : files,
        'labels': dblabels_as_string,
        'scores': scores,
        'colors': [cmap_dict['Base']]*len(x) # See update_colors method for comments
    })

    # Set up widgets
    button = Button(label='Plot Selected')
    button.on_click(update_plot)

    select = Select(title="Color Scheme:", value="Base", options=["Base", "Outliers", "Scores - Linear", "Scores - Log"])
    select.on_change('value',update_colors)

    # create a new plot and add a renderer
    left = figure(tools=TOOLS, plot_width=1000, plot_height=600, title=None)

    scatter = left.circle(x='x', y='y', fill_color='colors', line_color=None, size=4, source=s1)
    left.on_event(Tap,update_plot)
    plc = figure(plot_width=1000,plot_height=300,x_range=(0,10),y_range=(0,3))
    plc.xaxis.axis_line_color = None
    plc.yaxis.axis_line_color = None
    plc.xaxis.major_tick_line_color = None
    plc.xaxis.minor_tick_line_color = None
    plc.yaxis.major_tick_line_color = None
    plc.yaxis.minor_tick_line_color = None
    plc.xaxis.major_label_text_color = None
    plc.yaxis.major_label_text_color = None
    lc_path = 'Plots/'+s1.data['files'][0][:-5]+'.png'
    lc_image = plc.image_url(url=[lc_path],x=0,y=0,w=10,h=3,anchor="bottom_left")
    
    # Set up layouts and add to document
    inputs = row(button,select)
    #p = gridplot([[left,right]]) #future planning
    layout = column(inputs, left,plc)
    doc.add_root(layout)
    
show(modify_doc,notebook_url="http://localhost:8888")

In [27]:
plc = figure(plot_width=1000,plot_height=300,x_range=(0,10),y_range=(0,3))
plc.xaxis.axis_line_color = None
plc.yaxis.axis_line_color = None
plc.xaxis.major_tick_line_color = None
plc.xaxis.minor_tick_line_color = None
plc.yaxis.major_tick_line_color = None
plc.yaxis.minor_tick_line_color = None
plc.xaxis.major_label_text_color = None
plc.yaxis.major_label_text_color = None
plc.image_url(url=[],x=0,y=0,w=10,h=3,anchor="bottom_left")
show(p)

In [43]:
# sometimes it's hard to tell if bokeh is updating or stuck, running a dummy cell will prompt errors sometimes
True

True

In [105]:
p = figure()
source = ColumnDataSource(data={
    'x'      : [[0,1],[0,1],[0,1,2]],
    'y'      : [[0,1],[1,0],[.5,.5,.5]],
    'colors' : Category10[10][:3],
})
lp = p.multi_line('x','y',color='colors',source=source)
show(p)

In [58]:
import pickle
import sys
sys.path.append('python')
import clusterOutliers
import numpy as np
from quarterTools import colors_for_plot
import pandas as pd
from lightkurve import search_lightcurvefile

from bokeh.io import curdoc
from bokeh.plotting import figure
from bokeh.layouts import row,gridplot,column
from bokeh.models import ColumnDataSource, Circle
from bokeh.models.widgets import Button, Select
from bokeh.events import Tap
from bokeh.palettes import Colorblind4,Category10
from bokeh.transform import factor_cmap, linear_cmap, log_cmap

def modify_doc(doc):
    
    ##### Data Import #####
    with open('data/output/Q8_sample.coo','rb') as file:
        Q8_coo = pickle.load(file)

    data               = Q8_coo.data
    Quarter            = 8
    pca                = Q8_coo.reductions['PCA90']
    x                  = pca.iloc[:,0]
    y                  = pca.iloc[:,1]
    files              = data.index
    labels             = np.array(['KIC '+str(int(lc[4:13])) for lc in files])
    dblabels           = Q8_coo.scores['DB_outliers']
    dbcolors           = colors_for_plot(dblabels,'color_blind')
    dblabels_as_string = np.array(dblabels,dtype=str)
    scores             = Q8_coo.scores['k1_1000x10']
    lin_colors         = colors_for_plot(-scores,'viridis')
    log_colors         = colors_for_plot(-np.log(scores),'viridis')
    

    ##### Bokeh Setup #####
    # Tools to use
    TOOLS     = "pan,wheel_zoom,reset,tap,box_select,poly_select"
    palette   = Colorblind4[0], Colorblind4[1], Colorblind4[3] # Color palette for outlier cmap
    alphas    = {1:1,2:.9,3:.8,4:.75,5:.7,6:.65,7:.6,8:.55,9:.5,10:.5}
    cmap_dict = {
        "Base"            : palette[0],
        "Outliers"        : [to_hex(c) for c in dbcolors],
        "Scores - Linear" : [to_hex(c) for c in lin_colors],
        "Scores - Log"    : [to_hex(c) for c in log_colors],
    }

    def update_colors(attrname, old, new):
        if select.value == "Base":
            s1.data['colors'] = [cmap_dict[select.value]]*len(s1.data['x'])
        else:
            s1.data['colors'] = cmap_dict[select.value]
        #scatter.fill_color = cmap
        #sg.fill_color = nsg.fill_color = cmap
        #scatter.selection_glyph.fill_color=cmap
        #scatter.nonselection_glyph.fill_color=cmap

    # Bokeh data
    # create a column data source for the plots to share
    s1 = ColumnDataSource(data={
        'x'     : x,
        'y'     : y,
        'desc'  : labels,
        'files' : files,
        'labels': dblabels_as_string,
        'scores': scores,
        'colors': [cmap_dict['Base']]*len(x)
    }) 

    # Set up widgets
    button = Button(label='Plot Selected')

    select = Select(title="Color Scheme:", value="Base", options=["Base", "Outliers", "Scores - Linear", "Scores - Log"])
    select.on_change('value',update_colors)

    # create a new plot and add a renderer
    left = figure(tools=TOOLS, plot_width=1000, plot_height=600, title=None)
    scatter = left.circle(x='x', y='y', fill_color='colors', line_color=None, size=4, source=s1)

    # Set up layouts and add to document
    inputs = row(button,select)
    #p = gridplot([[left,right]]) #future planning
    layout = column(inputs, left)
    doc.add_root(layout)
    
show(modify_doc,notebook_url="http://localhost:8888")

In [14]:
np.log(scores)

kplr011241837-2011073133259_llc.fits   -5.741142
kplr011821279-2011073133259_llc.fits   -5.522272
kplr006342144-2011073133259_llc.fits   -6.213453
kplr010355036-2011073133259_llc.fits   -5.773762
kplr007757721-2011073133259_llc.fits   -6.703617
                                          ...   
kplr011721462-2011073133259_llc.fits   -6.238801
kplr007499535-2011073133259_llc.fits   -6.529726
kplr008960179-2011073133259_llc.fits   -4.968737
kplr010861822-2011073133259_llc.fits   -6.603322
kplr008462852-2011073133259_llc.fits   -3.466326
Name: k1_1000x10, Length: 20000, dtype: float64

In [43]:
lin_colors         = colors_for_plot(scores)
log_colors         = colors_for_plot(np.log(scores))

In [ ]:
lin_colors

In [52]:
p = figure()
p.circle([0,1],[0,1],size=10,fill_color=['#00bc00','#cccccc'])
show(p)

array([[  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    ,   0.    ,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    ,   0.    ,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.3255,   0.    ],
       [  0.    , 188.325

In [ ]:
##### Data Import #####
with open('data/output/Q8_sample.coo','rb') as file:
    Q8_coo = pickle.load(file)

data               = Q8_coo.data
Quarter            = 8
pca                = Q8_coo.reductions['PCA90']
x                  = pca.iloc[:,0]
y                  = pca.iloc[:,1]
files              = data.index
labels             = np.array(['KIC '+str(int(lc[4:13])) for lc in files])
dblabels           = Q8_coo.scores['DB_outliers']
dblabels_as_string = np.array(dblabels,dtype=str)
scores             = Q8_coo.scores['k1_1000x10']

In [6]:
import numpy as np
import pandas as pd

In [12]:
data               = Q8_coo.data[:1000]
Quarter            = 8
pca                = Q8_coo.reductions['PCA90']
x                  = pca.iloc[:1000,0]
y                  = pca.iloc[:1000,1]
files              = ['./Plots/'+file[:-5]+'.png' for file in data.index]
labels             = np.array(['KIC '+str(int(lc[12:21])) for lc in files])
dblabels           = Q8_coo.scores['DB_outliers'][:1000]
scores             = Q8_coo.scores['k1_1000x10'][:1000]
dbcolors           = colors_for_plot(dblabels,'color_blind')
lin_colors         = colors_for_plot(-scores,'viridis')
log_colors         = colors_for_plot(-np.log(scores),'viridis')
cmap_dict = {
    "Outliers"        : [to_hex(c) for c in dbcolors],
    "Scores - Linear" : [to_hex(c) for c in lin_colors],
    "Scores - Log"    : [to_hex(c) for c in log_colors],
}

In [69]:
Category10[10][0]

'#1f77b4'

In [13]:
gui_df = pd.DataFrame(index=data.index,data={
    'x':x,
    'y':y,
    'files':files,
    'labels':labels,
    'dblabels':dblabels,
    'scores':scores,
    'Outliers':cmap_dict['Outliers'],
    'Scores - Linear':cmap_dict['Scores - Linear'],
    'Scores - Log':cmap_dict['Scores - Log'],
    'colors':['#1f77b4']*len(x)
})

In [17]:
from bokeh.io import reset_output
reset_output()

In [73]:
import pandas as pd

from bokeh.io import show, output_file
output_file = 'gui.html'
from bokeh.resources import CDN
from bokeh.embed import file_html
from bokeh.plotting import figure
from bokeh.layouts import row,gridplot,column
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Button, Select
from bokeh.events import Tap
from bokeh.palettes import Category10

gui_df = pd.read_csv('gui_df.csv')

def modify_doc(doc):
    
    s1 = ColumnDataSource(gui_df)
    def update_colors(attrname, old, new):
        """ 
        Bokeh was being uncooperative with it's color mapper combined with the selection tool.
        I could get the colors to change when a different color selection was made,
        but it wouldn't update the color mapping to selected data. Whenever switching to a new
        color selection, selecting data would revert to the previous color mapping except where
        colors are given explicitly. So, I'm defining the colors explicitly using matplotlib
        instead of using the more convenient Bokeh mapper function which I can't find a way to save the 
        mapping as an array for the life of me. It's clunky and I hate it but it works.
        """
        if select.value == "Base":
            s1.data['colors'] = [Category10[10][0]]*len(s1.data['x'])
        else:
            s1.data['colors'] = s1.data[select.value]
            
    def update_plot():
        try:
            ind = s1.selected.indices[0] # can only plot one at a time
        except:
            ind = 0
            
        lc_path = s1.data['files'][ind]
        lc_image = plc.image_url(url=[lc_path],x=0,y=0,w=10,h=3,anchor="bottom_left")
        
    # Set up widgets
    button = Button(label='Plot Selected')
    button.on_click(update_plot)

    select = Select(title="Color Scheme:", value="Base", options=["Base", "Outliers", "Scores - Linear", "Scores - Log"])
    select.on_change('value',update_colors)

    # create a new plot and add a renderer
    TOOLS     = "pan,wheel_zoom,reset,tap,box_select,poly_select"
    left = figure(tools=TOOLS, plot_width=1000, plot_height=600, title=None)

    scatter = left.circle(x='x', y='y', fill_color='colors', line_color=None, size=4, source=s1)
    left.on_event(Tap,update_plot)
    plc = figure(plot_width=1000,plot_height=300,x_range=(0,10),y_range=(0,3))
    plc.xaxis.axis_line_color = None
    plc.yaxis.axis_line_color = None
    plc.xaxis.major_tick_line_color = None
    plc.xaxis.minor_tick_line_color = None
    plc.yaxis.major_tick_line_color = None
    plc.yaxis.minor_tick_line_color = None
    plc.xaxis.major_label_text_color = None
    plc.yaxis.major_label_text_color = None
    lc_path = s1.data['files'][0]
    lc_image = plc.image_url(url=[lc_path],x=0,y=0,w=10,h=3,anchor="bottom_left")
    
    # Set up layouts and add to document
    inputs = row(button,select)
    #p = gridplot([[left,right]]) #future planning
    layout = column(inputs, left, plc)
    doc.add_root(layout)

html = file_html(modify_doc, CDN, "my plot")
show(modify_doc,notebook_url="http://localhost:8888")

In [16]:
gui_df.to_csv('gui_df.csv')